In [1]:
# Dependencies
from citipy import citipy as ct
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import random
import scipy.stats as sts
import gmaps
from config import weather_key, gkey

gmaps.configure(api_key=gkey)

In [2]:
# Get list of all unique cities available through citipy
coords = list(ct.WORLD_CITIES_DICT.keys())
city_list = list(set([ct.nearest_city(coords[index][0],coords[index][1]).city_name for index in range(len(coords))]))

In [3]:
# create weather dataframe
weather_data = pd.DataFrame(columns=['city','country','lat/lng','temp','hum','cld','wnd'])

# loop through a random sample of 650 cities and store weather information in dataframe
# some cities not available in weather api, so sample of 650 is used to compansate and get at least 500 in the dataframe
for city in random.sample(city_list,650):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=imperial&appid={weather_key}"
    response = requests.get(url).json()
    try:
        weather_data = weather_data.append({'city': city,
                                            'country': response['sys']['country'],
                                            'lat/lng': [response['coord']['lat'], response['coord']['lon']],
                                            'temp': response['main']['temp'],
                                            'hum': response['main']['humidity'],
                                            'cld': response['clouds']['all'],
                                            'wnd': response['wind']['speed']},ignore_index=True)
    except KeyError:
        # annotate which cities were skipped over
        print(f"No weather information found for city: {city}")

# confirm number of cities in final dataframe
print(f"Samples pulled: {len(weather_data)}")

# clean datatype for the two columns
weather_data[['hum','cld']] = weather_data[['hum','cld']].astype(float)

weather_data.head()

No weather information found for city: qurunbaliyah
No weather information found for city: rajamahendri
No weather information found for city: ondal
No weather information found for city: guljanci
No weather information found for city: chore
No weather information found for city: alipur duar
No weather information found for city: inabaan
No weather information found for city: amilpas
No weather information found for city: drajna
No weather information found for city: guichicovi
No weather information found for city: crab hill
No weather information found for city: churumuco
No weather information found for city: brajrajnagar
No weather information found for city: valea macrisului
No weather information found for city: bolshoy uluy
No weather information found for city: kariotissa
No weather information found for city: ixtacuixtla
No weather information found for city: addis zemen
No weather information found for city: bad sankt leonhard
No weather information found for city: nestorion


,city,country,lat/lng,temp,hum,cld,wnd
0,kunfeherto,HU,"[46.3609, 19.4145]",64.40,29.0,0.0,19.57
1,kirovsk,UA,"[48.6375, 38.6428]",45.45,41.0,34.0,2.46
2,chitipa,MW,"[-9.7024, 33.2697]",62.55,97.0,24.0,4.52
3,juwana,ID,"[-6.715, 111.1514]",76.24,88.0,46.0,4.09
4,davie,US,"[26.0629, -80.2331]",89.46,55.0,40.0,11.50


In [4]:
vacation = weather_data.loc[(weather_data.cld==0)&((weather_data.temp<80) & (weather_data.temp>65))&(weather_data.wnd<10)]
vacation = vacation.reset_index(drop=True)
vacation

,city,country,lat/lng,temp,hum,cld,wnd
0,solin,HR,"[43.55, 16.5]",66.20,48.0,0.0,3.44
1,duki,PK,"[30.1541, 68.5728]",78.44,24.0,0.0,5.93
2,manastiur,RO,"[45.8167, 22.0667]",65.19,27.0,0.0,2.30
3,martina franca,IT,"[40.7031, 17.3339]",67.01,30.0,0.0,3.44
4,sesto calende,IT,"[45.7261, 8.634]",71.35,31.0,0.0,6.91
5,odzak,BA,"[45.0111, 18.3267]",66.20,30.0,0.0,4.61
6,zvornik,BA,"[44.3869, 19.1014]",66.20,32.0,0.0,9.22
7,rio cuarto,AR,"[-33.1307, -64.3499]",65.59,45.0,0.0,6.91
8,ozieri,IT,"[40.586, 9.0012]",68.81,52.0,0.0,8.05
9,annonay,FR,"[45.2397, 4.6708]",67.62,26.0,0.0,8.05


In [5]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = pd.DataFrame(columns=['Hotel Name','City','Country'])
for index,row in vacation.iterrows():
    params = {"location": str(row['lat/lng'])[1:-1],
              "radius": 5000,
              "type": 'hotel',
              "key": gkey}

    response = requests.get(url, params=params).json()
    name = response['results'][0]['name']
    hotels = hotels.append({'Hotel Name': name,
                   'City': row.city,
                   'Country': row.country},ignore_index=True)

hotels

,Hotel Name,City,Country
0,Split,solin,HR
1,Duki,duki,PK
2,Dumbrava,manastiur,RO
3,Martina Franca,martina franca,IT
4,Sesto Calende,sesto calende,IT
5,Modriča,odzak,BA
6,Zvornik,zvornik,BA
7,Río Cuarto,rio cuarto,AR
8,Ozieri,ozieri,IT
9,Annonay,annonay,FR


In [24]:
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(weather_data['lat/lng'],
                              weights=weather_data.hum,
                              max_intensity=200,
                              dissipating=False,
                              point_radius=3)
# info box code based off tutorial: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</d><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel) for hotel in hotels.to_dict(orient='records')]

marker_layer = gmaps.marker_layer(vacation['lat/lng'],info_box_content=hotel_info)
fig.add_layer(heatmap)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))